In [1]:
import os 

os.getcwd()
os.chdir('../')
os.getcwd()

'/home/gfspet/ml-projects/kidney-disease'

In [2]:
import sys
sys.path.append('/home/gfspet/ml-projects/kidney-disease/src')
from dataclasses import dataclass
from pathlib import Path 
from cnn_classifier.constants import * 
from cnn_classifier.utils.common import read_yaml, create_directories, save_json

import dagshub
dagshub.init(repo_owner='viansebastian', repo_name='Kidney-Disease-MLFlow-CICD', mlflow=True)

# os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/viansebastian/Kidney-Disease-MLFlow-CICD.mlflow'
# os.environ['MLFLOW_TRACKING_USERNAME']='viansebastian'
# os.environ['MLFLOW_TRACKING_PASSWORD']='c3aa83f40168a9b813c80d635a44343ef0a33451'
  
@dataclass(frozen=True)
class EvaluationConfig: 
    model_path: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_eval_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path='artifacts/training/final_model.keras',
            training_data='artifacts/data_ingestion/data',
            all_params=self.params,
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE, 
            mlflow_uri='https://dagshub.com/viansebastian/Kidney-Disease-MLFlow-CICD.mlflow'
        )
        
        return eval_config

[ 2024-11-24 15:28:33,232 ] 1038 httpx : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as viansebastian

[ 2024-11-24 15:28:33,241 ] 107 dagshub : INFO : helpers : Accessing as viansebastian
[ 2024-11-24 15:28:33,573 ] 1038 httpx : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/repos/viansebastian/Kidney-Disease-MLFlow-CICD "HTTP/1.1 200 OK"
[ 2024-11-24 15:28:33,838 ] 1038 httpx : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "viansebastian/Kidney-Disease-MLFlow-CICD"

[ 2024-11-24 15:28:33,843 ] 107 dagshub : INFO : helpers : Initialized MLflow to track repo "viansebastian/Kidney-Disease-MLFlow-CICD"


Repository viansebastian/Kidney-Disease-MLFlow-CICD initialized!

[ 2024-11-24 15:28:33,845 ] 107 dagshub : INFO : helpers : Repository viansebastian/Kidney-Disease-MLFlow-CICD initialized!


In [3]:
import tensorflow as tf
import numpy as np
import mlflow
import mlflow.keras
from mlflow.models.signature import infer_signature
from urllib.parse import urlparse

class Evaluator:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'nearest'
        )
        
        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.validation_generator = valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            shuffle=False,
            subset='validation',
            class_mode='categorical',
            **dataflow_kwargs
        )
    
    def evaluation(self): 
        self.model = self.load_model(self.config.model_path)
        self.valid_generator()
        self.score = self.model.evaluate(self.validation_generator)
        self.save_score()
        
    def save_score(self): 
        scores = {
            "loss" : self.score[0],
            "accuracy" : self.score[1]
        }
        save_json(
            path=Path('scores.json'),
            data=scores
        )
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_artifact_uri()).scheme
        
        if mlflow.active_run():
            mlflow.end_run()
        
        with mlflow.start_run(nested=True):
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {
                    "loss" : self.score[0],
                    "accuracy" : self.score[1]
                }
            )
            
            # input_example = np.random.random(1, *self.config.params_image_size)
            # signature = infer_signature(input_example)
            
            if tracking_url_type_store != 'file':
                mlflow.keras.log_model(
                    self.model, 
                    'model', 
                    registered_model_name='MobileNetV2Kidney',
                    # input_example=input_example,
                    # signature=signature
                )
            else: 
                mlflow.keras.log_model(
                    self.model, 
                    'model',
                    # input_example=input_example,
                    # signature=signature
                    )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)  
    
# try:
#     config = ConfigurationManager() 
#     eval_config = config.get_eval_config()
#     eval = Evaluator(config=eval_config) 
#     eval.evaluation()
#     eval.log_into_mlflow()
# except Exception as e: 
#     raise e

2024-11-24 15:28:34.145509: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 15:28:34.149510: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 15:28:34.159404: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732433314.175236  447920 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732433314.179402  447920 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 15:28:34.196831: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [4]:
try:
    config = ConfigurationManager() 
    eval_config = config.get_eval_config()
    eval = Evaluator(config=eval_config) 
    eval.evaluation()
    eval.log_into_mlflow()
except Exception as e: 
    raise e

[ 2024-11-24 15:28:36,577 ] 21 cnn_classifier_logger : INFO : common : yaml file: config/config.yaml loaded successfully
[ 2024-11-24 15:28:36,581 ] 21 cnn_classifier_logger : INFO : common : yaml file: params.yaml loaded successfully
[ 2024-11-24 15:28:36,582 ] 33 cnn_classifier_logger : INFO : common : Created directory at: artifacts


2024-11-24 15:28:36.591374: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Found 120 images belonging to 3 classes.


/home/gfspet/miniconda3/envs/kidney-disease/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 377ms/step - accuracy: 0.3822 - loss: 3.0708
[ 2024-11-24 15:28:43,582 ] 40 cnn_classifier_logger : INFO : common : JSON file saved at: scores.json
🏃 View run nebulous-fawn-942 at: https://dagshub.com/viansebastian/Kidney-Disease-MLFlow-CICD.mlflow/#/experiments/0/runs/1e1de112d4ff46f0b697d7cacef4dbc5
🧪 View experiment at: https://dagshub.com/viansebastian/Kidney-Disease-MLFlow-CICD.mlflow/#/experiments/0


2024/11/24 15:28:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/11/24 15:29:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'MobileNetV2Kidney' already exists. Creating a new version of this model...
2024/11/24 15:30:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MobileNetV2Kidney, version 3
Created version '3' of model 'MobileNetV2Kidney'.


🏃 View run masked-kit-402 at: https://dagshub.com/viansebastian/Kidney-Disease-MLFlow-CICD.mlflow/#/experiments/0/runs/2c97136c993c4f75bcb7faa07e90031e
🧪 View experiment at: https://dagshub.com/viansebastian/Kidney-Disease-MLFlow-CICD.mlflow/#/experiments/0
